In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
import numpy as np
import pandas as pd
import os

def make_dataframe():

    # 절대경로
    path = []
    # train, val, test data
    dataset_gubuns = []
    # abnormal or normal 구분
    labels_gubuns = []

    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            if '.png' in filename:
                file_path = dirname + '/' + filename
                path.append(file_path)
            if '/train/' in file_path:
                dataset_gubuns.append('train')
            elif '/val/' in file_path:
                dataset_gubuns.append('val')
            else:
                dataset_gubuns.append('test')

            if '/abnormal/' in file_path:
                labels_gubuns.append('abnormal')
            elif '/normal/' in file_path:
                labels_gubuns.append('normal')
    data_df = pd.DataFrame({'path': path, 'dataset': dataset_gubuns,'label': labels_gubuns})
    return data_df

In [30]:
pd.set_option('display.max_colwidth', 200)
data_df = make_dataframe()
print('data_df shape :' ,data_df.shape)
data_df.head(5)

data_df shape : (605, 3)


,path,dataset,label
0,/kaggle/input/output/val/normal/DALLíñE 2023-03-10 23.42.04 - photo of a part of car without blemish.png,val,normal
1,/kaggle/input/output/val/normal/DALLíñE 2023-03-10 23.34.47 - photo of a part of car without blemish.png,val,normal
2,/kaggle/input/output/val/normal/DALLíñE 2023-03-11 00.57.59 - photo of a part of car.png,val,normal
3,/kaggle/input/output/val/normal/DALLíñE 2023-03-11 14.41.37 - photo of part of a car.png,val,normal
4,/kaggle/input/output/val/normal/DALLíñE 2023-03-11 01.16.06 - a part of a car.png,val,normal


#### Sequence class로 작성

In [31]:
from tensorflow.keras.utils import Sequence
import sklearn 
import cv2


BATCH_SIZE = 32
IMAGE_SIZE = 512

 
class CDSequence(Sequence):
    def __init__(self, filenames, labels, batch_size=32, aug=None, shuffle=False):
    # image의 절대경로들
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        # albumentation 객체
        self.aug = aug
        self.shuffle = shuffle

        # 훈련 데이터의 경우
        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
    # 총 step의 갯수
        return len(self.labels) // self.batch_size

    def __getitem__(self, index):
    # 현재 인덱스를 기준으로 batch_size만큼 데이터를 가져옴
        meta_data = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # 훈련, 검증 데이터세트인 경우
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]

        # 불러온 meta_data를 np.array로 저장할 빈 공간을 생성
        image_batch = np.zeros((meta_data.shape[0], 512, 512, 3))
        for i in range(meta_data.shape[0]):
            # cv2는 이미지를 BGR로 불러오기 때문이 RGB로 바꾸어줌
            image = cv2.cvtColor(cv2.imread(meta_data[i]), cv2.COLOR_BGR2RGB)
            # 이미지의 크기가 전부 다르기 때문에 통일 시켜 주어야함
            image = cv2.resize(image, (512, 512))
            # augmentation이 있으면 적용
            if self.aug is not None:
                image = self.aug(image=image)['image']
            
            # 빈 이미지 배치에 최종 이미지를 등록
            image_batch[i] = image
        
        return image_batch, label_batch

    def on_epoch_end(self):
        # 파일과 라벨을 같이 섞어 주어야한다.
        if self.shuffle:
            self.image_filenames, self.labels = sklearn.utils.shuffle(self.filenames, self.labels)
        else:
            pass

In [32]:
import albumentations as A

train_df = data_df[data_df['dataset'] == 'train']
val_df = data_df[data_df['dataset'] == 'val']
test_df = data_df[data_df['dataset'] == 'test']

tr_path = train_df['path'].values
tr_label = pd.factorize(train_df['label'])[0]

val_path = val_df['path'].values
val_label = pd.factorize(val_df['label'])[0]

test_path = test_df['path'].values
test_label = pd.factorize(test_df['label'])[0]


augmentor_01 = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(scale_limit=(0.5, 0.9), p=0.5, rotate_limit=30),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
    A.Blur(p=0.2),
    
    
])

# CD_dataset = CDSequence(tr_path, tr_labels, batch_size=32, aug=augmentor_01, shuffle=False)
# show_first_data(CD_dataset, image_verbose=True)

print(tr_path[:5], tr_label[:5])

['/kaggle/input/output/train/normal/DALLíñE 2023-03-11 00.55.42 - photo of a part of car.png'
 '/kaggle/input/output/train/normal/DALLíñE 2023-03-11 01.04.49 - photo of a part of car.png'
 '/kaggle/input/output/train/normal/DALLíñE 2023-03-11 14.25.21 - part of a car.png'
 '/kaggle/input/output/train/normal/DALLíñE 2023-03-11 14.32.34 - part of a car.png'
 '/kaggle/input/output/train/normal/DALLíñE 2023-03-11 14.22.54 - part of a car.png'] [0 0 0 0 0]


In [33]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.xception import preprocess_input
import sklearn
import cv2

class CDSequence(Sequence):
    def __init__(self, filenames, labels, batch_size=32, aug=None, shuffle=False, pre_func=None):
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.aug = aug
        self.shuffle = shuffle
        self.pre_func = pre_func

        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        return len(self.labels) // self.batch_size

    def __getitem__(self, index):
        meta_data = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # 훈련, 검증 데이터세트인 경우
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]

        # 불러온 meta_data를 np.array로 저장할 빈 공간을 생성
        image_batch = np.zeros((meta_data.shape[0], 512, 512, 3), dtype='float32')
        for i in range(meta_data.shape[0]):
            # cv2는 이미지를 BGR로 불러오기 때문이 RGB로 바꾸어줌
            image = cv2.cvtColor(cv2.imread(meta_data[i]), cv2.COLOR_BGR2RGB)
            # 이미지의 크기가 전부 다르기 때문에 통일 시켜 주어야함
            image = cv2.resize(image, (512, 512))
            # augmentation이 있으면 적용
            if self.aug is not None:
                image = self.aug(image=image)['image']
            
            # 이미지 값을 self.pre_func 함수로 스케일링
            if self.pre_func is not None:
                image = self.pre_func(image)

            # 빈 이미지 배치에 최종 이미지를 등록
            image_batch[i] = image
        
        return image_batch, label_batch

    def on_epoch_end(self):
        # 파일과 라벨을 같이 섞어 주어야한다.
        if self.shuffle:
            self.filenames, self.labels = sklearn.utils.shuffle(self.filenames, self.labels)
        else:
            pass

In [34]:
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocess_input

tr_dataset = CDSequence(tr_path, tr_label, batch_size=32, aug=augmentor_01, shuffle=True, pre_func=eff_preprocess_input)
val_dataset = CDSequence(val_path, val_label, batch_size=32, aug=None, shuffle=False, pre_func=eff_preprocess_input)

In [35]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import Xception, ResNet50V2, EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
from tensorflow.keras.applications import MobileNet, MobileNetV2
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
import tensorflow as tf

def create_model(model_type='xception', verbose=False):
    # EfficientNet으로 한번 시도해보자. 안해본 모델도 모두 추가.
    
    input_tensor = Input(shape=(512, 512, 3))
    if model_type == 'resnet50v2':
        base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'xception':
        base_model = tf.keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb0':
        base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb1':
        base_model = tf.keras.applications.EfficientNetB1(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb2':
        base_model = tf.keras.applications.EfficientNetB2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb3':
        base_model = tf.keras.applications.EfficientNetB3(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb4':
        base_model = tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb5':
        base_model = tf.keras.applications.EfficientNetB5(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb6':
        base_model = tf.keras.applications.EfficientNetB6(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb7':
        base_model = tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet', input_tensor=input_tensor)
    
    bm_output = base_model.output
    x = GlobalAveragePooling2D()(bm_output)
    x = Dense(64, activation='relu', name='fc1')(x)
    x = Dropout(rate=0.5)(x)
    output = Dense(1, activation='sigmoid', name='output')(x)
    
    model = Model(input_tensor, output)
    
    if verbose:
        model.summary()
    return model

In [36]:
model = create_model(model_type='efficientnetb3', verbose=False)

In [37]:
print(f'모델의 레이어수: {len(model.layers)}')

모델의 레이어수: 389


#### 1. Effecient Net B0 model fine tuning

In [38]:
for layer in model.layers[:-4]:
    layer.trainable = False
for layer in model.layers[-4:]:
    layer.trainable = True

In [24]:
for idx, layer in enumerate(model.layers):
    if idx < 100:
        layer.trainable = False
    else:
        layer.trainable = True

In [39]:
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                patience=4,
                                verbose=1,
                                factor=0.2,
                                min_lr = 0.000001)
es = EarlyStopping(monitor='val_loss',
                  min_delta = 0,
                  patience=5,
                  verbose=1,
                  restore_best_weights=True)

In [40]:
tr_dataset

In [41]:
First_epochs = 100
second_epochs = 100

hist = model.fit(tr_dataset, epochs=First_epochs, validation_data=val_dataset,
                verbose=1, callbacks=[es, lr_reduction])

# for layer in model.layers:
#     if not isinstance(layer):
#         layer.trainable = True
# model.compile(optimizer=Adam(0.00001), loss='binary_crossentropy', metrics=['accuracy'])
# hist = model.fit(tr_dataset, epochs=second_epochs, validation_data=val_dataset,
#                 verbose=1, callbacks=[es, lr_reduction])



Epoch 1/100


2023-03-22 06:51:22.182547: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


11/11 [==============================] - 36s 2s/step - loss: 0.3863 - accuracy: 0.8409 - val_loss: 0.2358 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 2/100
11/11 [==============================] - 20s 2s/step - loss: 0.1646 - accuracy: 0.9489 - val_loss: 0.1564 - val_accuracy: 0.9479 - lr: 0.0010
Epoch 3/100
11/11 [==============================] - 20s 2s/step - loss: 0.1304 - accuracy: 0.9375 - val_loss: 0.1847 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 4/100
11/11 [==============================] - 21s 2s/step - loss: 0.0745 - accuracy: 0.9744 - val_loss: 0.1214 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/100
11/11 [==============================] - 21s 2s/step - loss: 0.0834 - accuracy: 0.9744 - val_loss: 0.1176 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/100
11/11 [==============================] - 20s 2s/step - loss: 0.0771 - accuracy: 0.9773 - val_loss: 0.1319 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 7/100
11/11 [==============================] - 21s 2s/step - loss: 0.0529 - accura

In [81]:
layers

NameError: name 'layers' is not defined

In [64]:
# hist = model.fit(tr_dataset, epochs=1000, validation_data=val_dataset,
#                 verbose=1, callbacks=[es, lr_reduction])

Epoch 1/1000


2023-03-22 05:36:31.622751: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


11/11 [==============================] - 48s 2s/step - loss: 0.3867 - accuracy: 0.8409 - val_loss: 0.2713 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 2/1000
11/11 [==============================] - 21s 2s/step - loss: 0.1806 - accuracy: 0.9460 - val_loss: 0.2532 - val_accuracy: 0.9688 - lr: 0.0010
Epoch 3/1000
11/11 [==============================] - 20s 2s/step - loss: 0.1064 - accuracy: 0.9716 - val_loss: 0.8223 - val_accuracy: 0.9271 - lr: 0.0010
Epoch 4/1000
11/11 [==============================] - 21s 2s/step - loss: 0.0871 - accuracy: 0.9716 - val_loss: 0.4350 - val_accuracy: 0.9479 - lr: 0.0010
Epoch 5/1000
11/11 [==============================] - 20s 2s/step - loss: 0.1105 - accuracy: 0.9659 - val_loss: 0.1682 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/1000
11/11 [==============================] - 20s 2s/step - loss: 0.0286 - accuracy: 0.9858 - val_loss: 0.1656 - val_accuracy: 0.9479 - lr: 0.0010
Epoch 7/1000
11/11 [==============================] - 21s 2s/step - loss: 0.0406 - 

#### efficient B0

In [65]:
test_dataset = CDSequence(test_path, test_label, batch_size=32, aug=None, shuffle=False, pre_func=eff_preprocess_input)
res = model.evaluate(test_dataset)
print('loss: ', res[0], 'accuracy: ', res[1])

3/3 [==============================] - 5s 2s/step - loss: 0.0868 - accuracy: 0.9896
loss:  0.08679777383804321 accuracy:  0.9895833134651184


In [85]:
test_dataset = CDSequence(test_path, test_label, batch_size=32, aug=None, shuffle=False, pre_func=eff_preprocess_input)
res = model.evaluate(test_dataset)
print('loss: ', res[0], 'accuracy: ', res[1])

3/3 [==============================] - 4s 1s/step - loss: 0.0379 - accuracy: 0.9792
loss:  0.03790709376335144 accuracy:  0.9791666865348816


In [27]:
test_dataset = CDSequence(test_path, test_label, batch_size=32, aug=None, shuffle=False, pre_func=eff_preprocess_input)
res = model.evaluate(test_dataset)
print('loss: ', res[0], 'accuracy: ', res[1])

3/3 [==============================] - 5s 2s/step - loss: 0.2230 - accuracy: 0.9583
loss:  0.22295747697353363 accuracy:  0.9583333134651184


#### EfficientNetB3
-   A.HorizontalFlip(p=0.5),
-   A.VerticalFlip(p=0.5),
-   A.ShiftScaleRotate(scale_limit=(0.5, 0.9), p=0.5, rotate_limit=30),
-   A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
-   A.Blur(p=0.2),

In [42]:
test_dataset = CDSequence(test_path, test_label, batch_size=32, aug=None, shuffle=False, pre_func=eff_preprocess_input)
res = model.evaluate(test_dataset)
print('loss: ', res[0], 'accuracy: ', res[1])

3/3 [==============================] - 5s 1s/step - loss: 0.0377 - accuracy: 0.9896
loss:  0.03771309554576874 accuracy:  0.9895833134651184
